In [29]:
# I wrote this as part of a job interview assignment where 2 VCF files are provided using two different variant callers 
# The task was to write an application in Python to merge these into a single VCF file, so that overlapping variants 
# are present only once in the output file.
# Two files were provided were - freebayes_raw.vcf and varscan_raw.vcf
# While merging we need to add a new tag in the INFO field of each variant, named ‘calledBy’ if the variant 
# is called by both tools. Need to set value as ‘calledBy=Freebayes+VarScan’.
# For the common variants, rename any common INFO and FORMAT tags annotated by both tools by prefixing the 
# tool name in the name of the tag. Ex: if ‘DP’ tag is annotated by both callers, rename the tags as 
# ‘Freebayes_DP’ and ‘VarScan_DP’ while merging.
# And the challenge was..."Do not use Pandas or any other vcf reader modules to read vcf files. Solve using core python data structures."


In [30]:
# Function to print lines provide a VCF file and a list of variant postions
def print_lines(fileName, posList):
    with open(fileName, 'r') as vcf:
        for line in vcf.readlines():
            if line.startswith("chr"):
                row = (line.rstrip("\n").split("\t"))
                for pos in posList:
                    if pos in row: 
                        print('\t'.join([str(v) for v in row])) 
    vcf.close()

In [31]:
import sys

f1 = open('freebayes_raw.vcf', 'r')
f2 = open('varscan_raw.vcf', 'r')

#Small hack for redirecting std out (print statements) to a file...
#orig_stdout = sys.stdout
#fs = open('Merged.vcf', 'w')
#sys.stdout = fs

header = "CHROM POS ID REF ALT QUAL FILTER INFO FORMAT GT".split()
print('\t'.join(header))

dict1={}
dict2={}
format_list1=[]
format_list2=[]
pos_list_f1=[]
pos_list_f2=[]
pos_list_common=[]
unique_f1=[]
unique_f2=[] 

# read the csv file
file1 = f1.readlines()
file2 = f2.readlines()

# adding file1 contents into dict1 and file 2 contents to dict2
# Separating the headers as keys and rows as values into a dictionary
for line1 in file1:    
    if line1.startswith ("#CHR"):
        h1 = list(line1.rstrip("\n").split("\t"))        
    if line1.startswith ('chr'):
        row = (line1.rstrip("\n").split("\t"))
        for item in range(len(h1)):
            dict1[h1[item]]=(row[item])  
        #print (dict1.keys())
        #print (dict1.values())

        #keeping track of all variant postions, assuming each variant position in each line is unique!
        #maintain a list of postions in f1 to check after reading entire file
        pos_list_f1.append(dict1['POS'])
        #print(pos_list_f1)                                        

        for line2 in file2:          
            if line2.startswith ("#CHR"):
                h2 = list(line2.rstrip("\n").split("\t"))
                #print (h2)
                #fs.write(line)
            if line2.startswith ('chr'):
                row = (line2.rstrip("\n").split("\t"))
                #store the row (line) values as items in a dictionary with headers as keys
                for item in range(len(h2)):
                    dict2[h2[item]]=(row[item])
                #collect the position list of file2    
                pos_list_f2.append(dict2['POS'])
              
                # considering only CHROM, POS, REF and ALT  in the two files to find the common variants 
                if (dict2['#CHROM']==dict1['#CHROM'] and dict2['POS']==dict1['POS']):
                    if (dict2['REF']==dict1['REF'] and dict2['ALT']==dict1['ALT']):
                        format_str1 = dict1['FORMAT']
                        format_list1 = list(format_str1.split(":"))
                        
                        format_str2 = dict2['FORMAT']
                        format_list2 = list(format_str2.split(":"))
        
                        #appending the strings to the common items in format_list_1 and format_list_2
                        append_caller_str1 = 'Varscan_'
                        append_caller_str2 = 'Freebayes_'
                
                        for i, n in enumerate(format_list2):
                            for x, t in enumerate(format_list1):
                                if n == t:
                                    format_list2[i] = append_caller_str2 + format_list2[i]
                                    format_list1[x] = append_caller_str1 + format_list1[x]
                
                        string1 = ':'.join(format_list1)
                        string2 = ':'.join(format_list2)
                                        
                        dict1['FORMAT']=string1
                        dict2['FORMAT']=string2  
                        
                        #maintain a list of positions in f2 to check after reading entire file 
                        pos_list_common.append(dict2['POS'])
                        
                        # add the calledBy string to INFO key 
                        new_str='calledBy=Freebayes+VarScan'
                        dict2['INFO'] = dict2['INFO'] + ';' + new_str
                        dict2['INFO'] = dict2['INFO'] +  ';' + dict1['INFO']
                        dict2['FORMAT'] = dict2['FORMAT'] +  ';' + dict1['FORMAT']
                        #dict2['ID'] = dict2['ID'] +  ';' + dict1['ID'] # nothing here just a . so dropping the merge
                        dict2['QUAL'] = dict2['QUAL'] +  ';' + dict1['QUAL']
                        dict2['FILTER'] = dict2['FILTER'] +  ';' + dict1['FILTER']
                        dict2['Sample1'] = dict2['Sample1'] +  ';' + dict1['unknown']
                        #print("The concatenated dictionary2 INFO: " + dict2['INFO'] )
                        #print("The concatenated dictionary2 FORMAT: " + dict2['FORMAT'] )
                        #print (f"{count}: Merged (common) variant is {dict2}")

                        print('\t'.join([str(v) for k,v in dict2.items()])) 
                        #count += 1 
                        break
                    #accounting for the condition where CHROM, POS are same but REF and ALT are different - hence different variant
                    elif(dict2['REF'] != dict1['REF'] and dict2['ALT'] != dict1['ALT']):
                        print('\t'.join([str(v) for k,v in dict1.items()]))
                        print('\t'.join([str(v) for k,v in dict2.items()]))
                        pos_list_common.append(dict2['POS'])
                        #print (f"{count}: Ref/Alt are unique in file1 for this variant : {dict1}")
                        #print (f"{count}: Ref/Alt are unique in file2 for this variant : {dict2}")
                        #count += 1
                    else:  
                        break

#finding the unique variants in file1 and file2 to add to the output
#assigning the list to a 'set' get rid of duplicated values while reading the file
unique_f1 = list(set(pos_list_f1)- set(pos_list_common))
unique_f2 = list(set(pos_list_f2) - set(pos_list_common))
#print(unique_f1)
#print(unique_f2)

#printing lines that are unique to file1 and file2 using a function 'print_lines'
print_lines('freebayes_raw.vcf', unique_f1)
print_lines('varscan_raw.vcf', unique_f2)

#sys.stdout = orig_stdout
#fs.close()
f1.close()
f2.close()

CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	GT
chr2	29415792	.	G	A	.;1101.66	PASS;.	ADP=89;WT=0;HET=1;HOM=0;NC=0;calledBy=Freebayes+VarScan;AB=0.494737;ABP=3.03316;AC=1;AF=0.5;AN=2;AO=47;CIGAR=1X;DP=95;DPB=95;DPRA=0;EPP=27.4509;EPPR=33.5919;GTI=0;LEN=1;MEANALT=1;MQM=60;MQMR=60;NS=1;NUMALT=1;ODDS=231.64;PAIRED=0.978723;PAIREDR=0.979167;PAO=0;PQA=0;PQR=0;PRO=0;QA=1807;QR=1719;RO=48;RPL=12;RPP=27.4509;RPPR=33.5919;RPR=35;RUN=1;SAF=47;SAP=105.07;SAR=0;SRF=48;SRP=107.241;SRR=0;TYPE=snp	Freebayes_GT:GQ:SDP:Freebayes_DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:ADF:ADR;Varscan_GT:Varscan_DP:DPR:RO:QR:AO:QA:GL	0/1:163:93:89:45:44:49.44%:4.4109E-17:37:18:45:0:44:0;0/1:95:95,47:48:1719:47:1807:-134.266,0,-126.33
chr2	29416037	.	C	G	.;15878.5	PASS;.	ADP=1305;WT=0;HET=1;HOM=0;NC=0;calledBy=Freebayes+VarScan;AB=0.45884;ABP=23.567;AC=1;AF=0.5;AN=2;AO=641;CIGAR=1X;DP=1397;DPB=1397;DPRA=0;EPP=153.831;EPPR=38.447;GTI=0;LEN=1;MEANALT=2;MQM=60;MQMR=59.9735;NS=1;NUMALT=1;ODDS=3656.17;PAIRED=0.99532;PAIREDR=0.99470